In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output
import tkinter as tk
from tkinter import filedialog
import zipfile

# ====================
# Data Loading using Tkinter
# ====================
root = tk.Tk()
root.withdraw()  # Hide the root Tk window
root.attributes("-topmost", True)

# Define file keys for CSV files.
files = {
    "latitude": None,
    "longitude": None,
    "vibration1": None,
    "vibration2": None,
    "speed": None
}

def load_file(key):
    file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if file_path:
        files[key] = file_path
        print(f"{key.capitalize()} file loaded: {file_path}")

print("Select Latitude File")
load_file("latitude")
print("Select Longitude File")
load_file("longitude")
print("Select Vibration 1 File")
load_file("vibration1")
print("Select Vibration 2 File")
load_file("vibration2")
print("Select Speed File")
load_file("speed")

# Load each CSV into a DataFrame and add a 'timestamp' using the row index.
dataframes = {}
for key, file_path in files.items():
    if file_path:
        df = pd.read_csv(file_path, header=None, names=[key])
        df['timestamp'] = df.index
        dataframes[key] = df
    else:
        print(f"{key.capitalize()} file not selected.")

# ====================
# Create GPS DataFrame by merging latitude and longitude.
# ====================
if "latitude" in dataframes and "longitude" in dataframes:
    df_gps = pd.merge(dataframes["latitude"], dataframes["longitude"], on="timestamp")
    # Rename columns for consistency
    df_gps = df_gps.rename(columns={"latitude": "Latitude", "longitude": "Longitude"})
    # Add an index column for use in the interactive plot
    df_gps["PointIndex"] = df_gps.index

    # GPS clean up
    df_gps = df_gps.dropna(subset=["Latitude", "Longitude"])  # Remove rows with NaN values
    df_gps = df_gps[(df_gps["Latitude"] >= -90) & (df_gps["Latitude"] <= 90)]  # Latitude must be between -90 and 90
    df_gps = df_gps[(df_gps["Longitude"] >= -180) & (df_gps["Longitude"] <= 180)]  # Longitude must be between -180 and 180
    df_gps = df_gps.reset_index(drop=True)  # Reset index after filtering
    df_gps["PointIndex"] = df_gps.index  # Update PointIndex after filtering

else:
    print("Latitude or Longitude data is missing.")
    df_gps = pd.DataFrame(columns=["Latitude", "Longitude", "PointIndex"])

# ====================
# Merge the two vibration signals on 'timestamp'
# ====================
if "vibration1" in dataframes and "vibration2" in dataframes:
    df_vibration_merged = pd.merge(
        dataframes["vibration1"],
        dataframes["vibration2"],
        on="timestamp"
        # When the column names differ (here: vibration1 vs vibration2), suffixes are not needed.
    )
    # You may rename columns if desired; here they remain "vibration1" and "vibration2"
else:
    print("Vibration data files are missing.")
    df_vibration_merged = pd.DataFrame()

# ====================
# Data Preprocessing and Segmentation for Vibration Data #come back to this if needed
# ====================
dt_vibration = 0.002  # seconds per sample (e.g. 500 Hz sampling rate)
segment_duration_seconds = 10
segment_length = int(segment_duration_seconds / dt_vibration)
if not df_vibration_merged.empty:
    num_segments = len(df_vibration_merged) // segment_length
    segments = []
    for i in range(num_segments):
        seg = df_vibration_merged.iloc[i * segment_length: (i + 1) * segment_length][["vibration1", "vibration2"]].values
        segments.append(seg)
    segments = np.array(segments)
    print("Segmented vibration data shape:", segments.shape)
else:
    segments = np.array([])
    print("No vibration data available for segmentation.")

# ====================
# Build the Interactive Dash App
# ====================

# Create the interactive GPS map using Plotly Express.
if not df_gps.empty and len(df_gps) > 0:
    max_vib_points = min(len(df_gps), len(segments))
    gps_vib = df_gps.iloc[:max_vib_points].copy()
    # Use custom_data to store the point index so that it will be available in callbacks.
    map_fig = px.scatter_mapbox(
        gps_vib,
        lat="Latitude",
        lon="Longitude",
        custom_data=["PointIndex"],
        zoom=10,
        title="GPS Points with Vibration Data"
    )
    map_fig.update_layout(mapbox_style="open-street-map", height=600)
else:
    map_fig = go.Figure()
    map_fig.update_layout(title="No GPS Data Available", height=600)

# clickable vibration points
if len(gps_vib) > 0:
    map_fig.add_trace(go.Scattermapbox(
        lat=gps_vib["Latitude"],
        lon=gps_vib["Longitude"],
        mode="markers",
        marker=dict(size=8, color="hotpink"),
        customdata=[[i] for i in range(len(gps_vib))], 
        hovertext=["Point Index: {}".format(idx) for idx in gps_vib["PointIndex"]]
    ))

# # Create an initial empty vibration plot figure.
# vib_empty_fig = go.Figure()
# vib_empty_fig.update_layout(
#     title="Vibration Signal",
#     xaxis_title="Time (s)",
#     yaxis_title="Acceleration"
# )

    map_fig.update_layout(
        mapbox_style="open-street-map",
        mapbox=dict(
            center=dict(
                lat=gps_vib.iloc[0]["Latitude"], 
                lon=gps_vib.iloc[0]["Longitude"]
            ),
            zoom=12
        ),
        height=600,
        title=f" GPS Track with {len(gps_vib)} Vibration Points",
        showlegend=True
    )
    
else:
    map_fig = go.Figure()
    map_fig.update_layout(title="No GPS Data Available", height=600)

# ========== Default Vibration Plot ==========
def create_vibration_plot(segment_index=0):
    """Create vibration plot for a specific segment"""
    if segments.size == 0:
        empty_fig = go.Figure()
        empty_fig.update_layout(
            title="No Vibration Data Available",
            xaxis_title="Time (s)",
            yaxis_title="Acceleration"
        )
        return empty_fig

    # Ensure segment index is valid
    if segment_index >= len(segments):
        segment_index = len(segments) - 1
    
    selected_segment = segments[segment_index]
    time_axis = np.arange(segment_length) * dt_vibration

    vib_fig = go.Figure()
    vib_fig.add_trace(go.Scatter(
        x=time_axis,
        y=selected_segment[:, 0],
        mode='lines',
        name='Vibration Channel 1'
    ))
    vib_fig.add_trace(go.Scatter(
        x=time_axis,
        y=selected_segment[:, 1],
        mode='lines',
        name='Vibration Channel 2'
    ))
    vib_fig.update_layout(
        title=f"Vibration Signal for GPS Point {segment_index}",
        xaxis_title="Time (s)",
        yaxis_title="Acceleration"
    )
    return vib_fig

# Create default vibration plot
default_vib_fig = create_vibration_plot(0)

# Initialize Dash app.
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Graph(id="gps-map", figure=map_fig)
    ], style={'width': '48%', 'display': 'inline-block', 'vertical-align': 'top'}),
    html.Div([
        dcc.Graph(id="vibration-plot", figure=default_vib_fig)
    ], style={'width': '48%', 'display': 'inline-block', 'vertical-align': 'top'})
])


# --------------------
# Callback to Update the Vibration Plot Based on Clicked GPS Point
# --------------------
@app.callback(
    Output('vibration-plot', 'figure'),
    [Input('gps-map', 'clickData')]
)
def update_vibration_plot(clickData):
    # If no point is selected, return the empty vibration figure.
    if clickData is None:
        return default_vib_fig

    if segments.size == 0:
        empty_fig = go.Figure()
        empty_fig.update_layout(
            title="No Vibration Data Available",
            xaxis_title="Time (s)",
            yaxis_title="Acceleration"
        )
        return default_vib_fig

    try:
        clicked_point = clickData['points'][0]
        # Extract the segment index from customdata
        if 'customdata' in clicked_point:
            segment_index = clicked_point['customdata'][0]  # Get first element of the array
        else:
            return default_vib_fig
            
    except (KeyError, IndexError, TypeError) as e:
        return default_vib_fig

    # Validate segment index
    if segment_index >= len(segments):
        segment_index = len(segments) - 1

    return create_vibration_plot(segment_index)
# ====================
# Run the Dash App
# ====================
if __name__ == "__main__":
    app.run(debug=True, port=8060)

Select Latitude File
Latitude file loaded: C:/Users/natal/Downloads/LLA class/Coding Assignments/Assignment4/Data 2/GPS.latitude.csv
Select Longitude File
Longitude file loaded: C:/Users/natal/Downloads/LLA class/Coding Assignments/Assignment4/Data 2/GPS.longitude.csv
Select Vibration 1 File
Vibration1 file loaded: C:/Users/natal/Downloads/LLA class/Coding Assignments/Assignment4/Data 2/CH1_ACCEL1Z1.csv
Select Vibration 2 File
Vibration2 file loaded: C:/Users/natal/Downloads/LLA class/Coding Assignments/Assignment4/Data 2/CH2_ACCEL1Z2.csv
Select Speed File
Speed file loaded: C:/Users/natal/Downloads/LLA class/Coding Assignments/Assignment4/Data 2/GPS.speed.csv
Segmented vibration data shape: (7199, 5000, 2)


C:\Users\natal\AppData\Local\Temp\ipykernel_6516\2721106647.py:118: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  map_fig = px.scatter_mapbox(
C:\Users\natal\AppData\Local\Temp\ipykernel_6516\2721106647.py:133: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

